<a href="https://colab.research.google.com/github/chengwailei/webscraper/blob/master/WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code allow scraping of the indication, COVID-19 trial, and drug developer information from an excel file containing the DB code of candidate compounds**


Source of indication: 

DrugBank (https://www.drugbank.ca/)
ChEMBL API (https://github.com/chembl/chembl_webresource_client)


Source of COVID-19 CT: 

ClinicalTrials.gov (https://clinicaltrials.gov/ct2/results?cond=COVID-19)


Source of COVID-19 paper: 

COVID-19 Crowd Generated Gene and Drug Set Library (https://amp.pharm.mssm.edu/covid19/)


Source of developer: 

Drug (https://www.drugs.com/pharmaceutical-companies.html

Source of route:

For ChEMBLID: INXIGHT:DRUGS (https://drugs.ncats.io/substances)

For DB ID: DrugBank (https://www.drugbank.ca/)

House Keeping

In [2]:
## Install required packages
!pip install requests
!pip install openpyxl
!pip install chembl_webresource_client
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install xlsxwriter
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for chembl-webresource-client: filename=chembl_webresource_client-0.10.1-cp36-none-any.whl size=57153 sha256=aa39feb04c3deb91ae4a94cbe28d2715c04f8042df2fb9d0906cf29cf99745dd
  Stored in directory: /root/.cache/pip/wheels/81/8e/3b/4ec9940a01673307821600bfac28b17971caf84ff2b64653cb
Successfully built chembl-webresource-client
     |████████████████████████████████| 911kB 2.7MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/re

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import openpyxl
import os
import re
from bs4 import NavigableString, Tag

Make a Pharmaceutical Company List based on Drugs.com

In [0]:
## Extract Pharmaceutical List from Drug.com
import requests
pharm_url = requests.get("https://www.drugs.com/pharmaceutical-companies.html").text
pharmsoup = BeautifulSoup(pharm_url)
pharmList = []
for pharm in pharmsoup.find_all('a',href=re.compile(r"/manufacturer/")):
    pharm1 = pharm.text
    pharm2 = pharm1.replace(",","")
    pharms = pharm2.replace(".","")
    pharmList.append(pharms.lower())
# print(pharmList)

Extract COVID 19 Clinical Trial Table from Clinicaltrials.gov

= Change max page if necessary

While loop doesn't work well with Selenium

= Alternative codes (expanded from while loop available below)

In [0]:
# file2 = '/content/output_IP.csv'
# clinical_trials = pd.read_csv(file2)
# ## COVID19 Clinical Trial List from clinicaltrials.gov
# from selenium import webdriver
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.support.select import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# capa = DesiredCapabilities.CHROME
# driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
# driver.set_window_size(1440,900)
# driver.get("https://clinicaltrials.gov/ct2/results?cond=COVID-19")
# page=1
# max_page=12
# Num=[]
# Status=[]
# StudyTitle=[]
# StudyLink=[]
# Conditions=[]
# Interventions=[]

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[2]/label/select/option[4]").click()


# while page<=max_page:

#   rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
#   for row in rows:
#     Num.append(row.find_element_by_xpath('./td[1]').text)
#     Status.append(row.find_element_by_xpath('./td[3]').text)
#     StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#     StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#     Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#     Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

#     driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
#     time.sleep(60)
#     page=page+1
#     print('navigate to page: ' + str(page))

# driver.close()

# df=pd.DataFrame({"Number":Num,"Interventions":Interventions,"Status":Status,"Study Title":StudyTitle,"Study Link":StudyLink,"Conditions":Conditions})

# print(df)
# #df.to_csv('output_IP.csv',index=False)

In [0]:
# ## COVID19 Clinical Trial List from clinicaltrials.gov Exapnded code
# from selenium import webdriver
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.support.select import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# capa = DesiredCapabilities.CHROME
# driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
# driver.set_window_size(1440,900)
# driver.get("https://clinicaltrials.gov/ct2/results?cond=COVID-19")
# page=1
# # max_page=1
# max_page=15
# Num=[]
# Status=[]
# StudyTitle=[]
# StudyLink=[]
# Conditions=[]
# Interventions=[]

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[2]/label/select/option[4]").click()


# # while page<=max_page:
# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# rows= WebDriverWait(driver, 10000).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@id='theDataTable']/tbody//tr")))
# for row in rows:
#   Num.append(row.find_element_by_xpath('./td[1]').text)
#   Status.append(row.find_element_by_xpath('./td[3]').text)
#   StudyTitle.append(row.find_element_by_xpath('./td[4]').text)
#   StudyLink.append(row.find_element_by_xpath('./td[4]/a').get_attribute('href'))
#   Conditions.append(row.find_element_by_xpath('./td[5]').get_attribute('textContent'))
#   Interventions.append(row.find_element_by_xpath('./td[6]').get_attribute('textContent'))

# driver.find_element_by_xpath("/html/body/div[6]/div[3]/div[2]/div[1]/div/div[2]/div[2]/div[6]/a[3]").click()
# time.sleep(60)
# page=page+1
# print('navigate to page: ' + str(page))

# driver.close()

# df=pd.DataFrame({"Number":Num,"Interventions":Interventions,"Status":Status,"Study Title":StudyTitle,"Study Link":StudyLink,"Conditions":Conditions})

# print(df)
# df.to_csv('output_IP.csv',index=False)

In [0]:
## COVID Trial Table Cleaning
file2 = './output_IP.csv'
clinical_trials = pd.read_csv(file2)
df1=clinical_trials
my_str = df1['Interventions'].str
substr = "Drug:"
substr1 = "Other"
substr2 = "Device"
substr3 = "Biological"
substr4 = "Diagnostic"
substr5 = "Procedure"
substr6 = "Placebo:"
inserttxt = "+"
df1['Interventions']=df1['Interventions'].str.replace(substr, inserttxt + substr)
df1['Interventions']=df1['Interventions'].str.replace(substr1, inserttxt + substr1)
df1['Interventions']=df1['Interventions'].str.replace(substr2, inserttxt +substr2)
df1['Interventions']=df1['Interventions'].str.replace(substr3, inserttxt +substr3)
df1['Interventions']=df1['Interventions'].str.replace(substr4, inserttxt +substr4)
df1['Interventions']=df1['Interventions'].str.replace(substr5, inserttxt +substr5)
df1['Interventions']=df1['Interventions'].str.replace(substr6, inserttxt +substr6)
df1['Interventions']=df1['Interventions'].str.replace('+', '', 1)
df1=df1.assign(var1=df1['Interventions'].str.split('+')).explode('var1')
df1['Status']=df1['Status'].str.replace("\nNEW","")
df1.Status.unique()
df1=df1[~df1.Status.str.contains("No longer available")]
df1=df1[~df1.Status.str.contains("Suspended")]
df1=df1[~df1.Status.str.contains("Terminated")]
df1 = df1[df1.Interventions.notnull()]
df1 = df1[['var1', 'Interventions', 'Study Title', 'Study Link', 'Status','Conditions']]
df1.columns = ['Drug', 'Interventions', 'Study Title', 'Study Link', 'Status','Conditions']
df1['COVID19_Trials'] = df1[['Interventions','Study Title','Study Link']].astype(str).apply(lambda x: '\n '.join(x), axis = 1)
df1['Drug']=df1['Drug'].str.replace('Drug: ','')
Clinical_trials = df1

Search Functions

In [0]:
## COVID19 Clinical Trials Search
def trialsearch(keyword):
  keyword1=keyword.replace(r"\(.*\)","")
  result=Clinical_trials.loc[Clinical_trials['Drug'].str.contains(keyword1), 'COVID19_Trials'].astype(str)
  # .str.contains('|'.join(substr), regex=True)]
  result= ' |\n'.join(result)
  if (result==""):
    return "No current trial"
  else:
    return result

In [0]:
# #In-vitro study search
# from selenium import webdriver
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.support.select import Select
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver import ActionChains
# from selenium.common.exceptions import NoSuchElementException

# def invitrostudysearch(keyword):
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless')
#     chrome_options.add_argument('--no-sandbox')
#     chrome_options.add_argument('--disable-dev-shm-usage')
#     capa = DesiredCapabilities.CHROME
#     driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
#     driver.set_window_size(1440,900)
#     driver.get("https://amp.pharm.mssm.edu/covid19/")

#     StudyName = []
#     StudyLink = []

#     # driver.find_element_by_id("pills-experimental-tab").click();
#     driver.find_element(By.XPATH,"//*[@id='experimental_table_filter']/label/input").click()
#     driver.implicitly_wait(2)
#     input_element = driver.find_element_by_css_selector('#experimental_table_filter > label > input[type=search]')
#     action = ActionChains(driver)
#     driver.implicitly_wait(2)
#     action.move_to_element(input_element)
#     action.click(input_element)
#     action.send_keys(keyword).perform()
#     page=1
#     max_page=4

#     while page<=max_page:
#       rows= WebDriverWait(driver, 40).until(EC.visibility_of_all_elements_located((By.XPATH, "/html/body/div[1]/div[4]/div/div/div[1]/div/div[1]/div/div/table/tbody/tr")))

#       for row in rows:
#           try:
#             elem=row.find_element_by_xpath('./td[2]')
#             driver.implicitly_wait(2)
#           except NoSuchElementException:
#             pass
#           try:
#             StudyName.append(row.find_element_by_xpath('./td[2]').text)
#             driver.implicitly_wait(2)
#             content=row.find_element_by_xpath('./td[2]/div').get_attribute('data-content')  
#             #  content1=re.search('<a href="(.*)" target', content).group(1)      
#             if "<a href=" in content:
#               content1=re.search('<a href="(.*)" target', content).group(1)
#             else:
#               content1="No Source Available"
#             StudyLink.append(content1)
#           except NoSuchElementException:
#             pass
#       driver.find_element_by_id("experimental_table_next").click();
#       page=page+1

#     driver.close()
#     if len(StudyName)!= 0:
#       InvitroStudy=pd.DataFrame({"Study_Title":StudyName,"Study_Link":StudyLink})
#       InvitroStudy['Study_Link'] = '(' + InvitroStudy['Study_Link'].astype(str)
#       InvitroStudy['Study_Link'] = InvitroStudy['Study_Link'].astype(str)+')'
#       InvitroStudy = InvitroStudy.drop_duplicates(subset=['Study_Title']) 
#       InvitroStudy['Study'] = InvitroStudy[['Study_Title', 'Study_Link']].apply(lambda x: ''.join(x), axis=1)
#       InvitroStudy = InvitroStudy.drop(InvitroStudy.columns[[0, 1]], axis=1) 
#       Invitro=InvitroStudy['Study'].str.cat(sep=' |\n')
#     else:
#         Invitro=""
#     return Invitro

In [0]:
# #Insilico study search 
# from selenium import webdriver
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# from selenium.webdriver.support.select import Select    
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver import ActionChains
# from selenium.common.exceptions import NoSuchElementException

# def insilicostudysearch(keyword):
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--headless')
#     chrome_options.add_argument('--no-sandbox')
#     chrome_options.add_argument('--disable-dev-shm-usage')
#     capa = DesiredCapabilities.CHROME
#     driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
#     driver.set_window_size(1440,900)
#     driver.get("https://amp.pharm.mssm.edu/covid19/")

#     StudyName = []
#     StudyLink = []

#     driver.find_element_by_id("pills-computational-tab").click();
#     driver.implicitly_wait(2)    
#     driver.find_element(By.XPATH,"//*[@id='experimental_table_filter']/label/input").click()
#     driver.implicitly_wait(2)
#     input_element = driver.find_element_by_css_selector('#computational_table_filter > label > input[type=search]')
#     driver.implicitly_wait(2)    
#     action = ActionChains(driver)
#     driver.implicitly_wait(2)
#     action.move_to_element(input_element)
#     driver.implicitly_wait(2)
#     action.click(input_element)
#     driver.implicitly_wait(2)    
#     action.send_keys(keyword).perform()
#     driver.implicitly_wait(2)    
#     page=1
#     max_page=4

#     while page<=max_page:
#       rows= WebDriverWait(driver, 40).until(EC.visibility_of_all_elements_located((By.XPATH, "/html/body/div[1]/div[4]/div/div/div[1]/div/div[2]/div/div/table/tbody//tr")))

#       for row in rows:
#           try:
#             elem=row.find_element_by_xpath('./td[2]')
#             driver.implicitly_wait(2)
#           except NoSuchElementException:
#             pass
#           try:
#              StudyName.append(row.find_element_by_xpath('./td[2]').text)
#              driver.implicitly_wait(2)
#              content=row.find_element_by_xpath('./td[2]/div').get_attribute('data-content')
#              driver.implicitly_wait(2)            
#              content1=re.search('<a href="(.*)" target', content).group(1)
#              driver.implicitly_wait(2)    
#              StudyLink.append(content1)
#           except NoSuchElementException:
#             pass
      
#       driver.find_element_by_id("computational_table_next").click();
#       page=page+1

#     driver.close()
#     if len(StudyName)!= 0:
#       InsilicoStudy=pd.DataFrame({"Study_Title":StudyName,"Study_Link":StudyLink})
#       InsilicoStudy['Study_Link'] = '(' + InsilicoStudy['Study_Link'].astype(str)
#       InsilicoStudy['Study_Link'] = InsilicoStudy['Study_Link'].astype(str)+')'
#       InsilicoStudy = InsilicoStudy.drop_duplicates(subset=['Study_Title']) 
#       InsilicoStudy['Study'] = InsilicoStudy[['Study_Title', 'Study_Link']].apply(lambda x: ''.join(x), axis=1)
#       InsilicoStudy = InsilicoStudy.drop(InsilicoStudy.columns[[0, 1]], axis=1) 
#       Insilico=InsilicoStudy['Study'].str.cat(sep=' |\n')
#     else:
#         Insilico=""
#     return Insilico

In [0]:
## ChEMBL Indication Search:
from chembl_webresource_client.new_client import new_client

def chemblindication(keyword):
  drug_indication = new_client.drug_indication
  mol_id = drug_indication.filter(molecule_chembl_id__icontains=format(keyword)).only(['mesh_heading', 'max_phase_for_ind'])
  chembldrugindi  = '-'.join([str(counterofobjects) for counterofobjects in mol_id])
  chembldrugindi.rsplit("}-{")
  chembldrugindiList1=chembldrugindi.replace("'mesh_heading':","")
  chembldrugindiList2=chembldrugindiList1.replace("'max_phase_for_ind': ","Phase")
  chembldrugindiList3=chembldrugindiList2.replace("'}-","|")
  indication1 = chembldrugindiList3.replace("{"," ")
  indication2 = indication1.replace("}"," ")
  indication = indication2.replace("'"," ")
  return indication

In [0]:
# # from chembl_webresource_client.new_client import new_client

# # def chemblindicationID(keyword):
#   drug_indication = new_client.drug_indication
#   mol_id = drug_indication.filter(molecule_chembl_id__icontains=format(keyword)).only(['mesh_id'])
#   chembldrugindi  = '-'.join([str(counterofobjects) for counterofobjects in mol_id])
#   chembldrugindi.rsplit("}-{")
#   chembldrugindiList1=chembldrugindi.replace("'mesh_id':","")
#   # chembldrugindiList2=chembldrugindiList1.replace("'max_phase_for_ind': ","Phase")
#   # chembldrugindiList3=chembldrugindiList2.replace("'}-","|")
#   # indication1 = chembldrugindiList3.replace("{"," ")
#   # indication2 = indication1.replace("}"," ")
#   # indication = indication2.replace("'"," ")
#   # return indication

In [0]:
## ChEMBL ID
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException

def searchchemblid(keyword):
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    capa = DesiredCapabilities.CHROME
    driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
    driver.set_window_size(1440,900)
    url = ("https://www.drugbank.ca/{}").format(keyword)
    driver.get(url)

    try:
        chemblID1=driver.find_element_by_partial_link_text('CHEMBL')
        chemblID1=chemblID1.get_attribute("href")
        chemblID=chemblID1.rpartition('/')[-1]
    except NoSuchElementException:
      chemblID=keyword
      pass
    driver.close()
    return(chemblID)

In [0]:
def comparer(L1, L2):
    for i in L2:
        for j in L1:
            if (i in j) or (j in i):
                yield j

In [0]:
## DB Developer Search
def searchdeveloper(keyword):
    from urllib.request import urlopen
    url = "https://www.drugbank.ca/drugs/{}".format(keyword)
    response = urlopen(url)
    soup = BeautifulSoup(response)

    manufacturer=[]

    for manlist in soup.find_all(attrs={"class":"manufacturer-columns"}):
      for man in manlist.find_all('li'):
        man1=man.text
        man2=man1.replace(",","")
        mans=man2.replace(".","")
        manufacturer.append(mans.lower())
    packager = []
    for packlist in soup.find_all(attrs={"class":"packager-columns"}):
      for pack in packlist.find_all('li'):
        pack1=pack.text
        pack2=pack1.replace(",","")
        packs=pack2.replace(".","")
        packager.append(packs.lower())
    manpack = manufacturer + packager
    manpackList = []
    for word in manpack:
      if word not in manpackList:
        manpackList.append(word)
    match1 = list(comparer(pharmList, manpackList))
    match = "|".join(str(x) for x in match1)
    return(match)
    # print(match)

In [0]:
from urllib.request import urlopen

def searchDBIndication(keyword):
  url = "https://www.drugbank.ca/drugs/{}".format(keyword)
  response = urlopen(url)
  soup = BeautifulSoup(response)
  indilist = []
  extr = soup.find('dt', string='Indication')
  indication = extr.find_next_sibling('dd').text
  return(indication)

In [0]:
from urllib.request import urlopen

def searchrouteDB(keyword):
  url = "https://www.drugbank.ca/drugs/{}".format(keyword)
  response = urlopen(url)
  soup = BeautifulSoup(response)
  indilist = []
  extr = soup.find('dt', string='Prescription Product')
  extr2= extr.find_next_sibling('table')

  return(extr2)

In [0]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

def searchrouteDB(keyword):
  url = "https://www.drugbank.ca/drugs/{}".format(keyword)
  routelist = []
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  extr = soup.find('dt', string='Prescription Products')
  if extr is not None:
    extr2 = extr.find_next_sibling('dd')
    extr3 = extr2.find_all('table')[0].tbody.find_all('tr')
    for row in extr3:
      information = row.find_all('td')[3].text
      routelist.append(information)
  else:
    extr = soup.find('dt', string='Over the Counter Products')
    if extr is not None:
      extr2 = extr.find_next_sibling('dd')
      extr3 = extr2.find_all('table')[0].tbody.find_all('tr')
      for row in extr3:
        information = row.find_all('td')[3].text
        routelist.append(information)  
    else:
      routelist=''
  routelist1 = []
  for word in routelist:
    if word not in routelist1:
      routelist1.append(word)
  route = "|".join(str(x) for x in routelist1)
  return(route)

In [0]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

def searchrouteCHEMBL(keyword):
  url = "https://www.ebi.ac.uk/chembl/assay_report_card/{}".format(keyword)
  routelist = []
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  extr = soup.find('dt', string='Prescription Products')
  if extr is not None:
    extr2 = extr.find_next_sibling('dd')
    extr3 = extr2.find_all('table')[0].tbody.find_all('tr')
    for row in extr3:
      information = row.find_all('td')[3].text
      routelist.append(information)
  else:
    extr = soup.find('dt', string='Over the Counter Products')
    if extr is not None:
      extr2 = extr.find_next_sibling('dd')
      extr3 = extr2.find_all('table')[0].tbody.find_all('tr')
      for row in extr3:
        information = row.find_all('td')[3].text
        routelist.append(information)  
    else:
      routelist=''
  routelist1 = []
  for word in routelist:
    if word not in routelist1:
      routelist1.append(word)
  route = "|".join(str(x) for x in routelist1)
  return(route)

In [0]:
#NIH route search 
import requests
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.select import Select    
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from urllib.request import urlopen

def searchrouteNIH(keyword):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    capa = DesiredCapabilities.CHROME
    driver = webdriver.Chrome('chromedriver',options=chrome_options, desired_capabilities=capa)
    driver.set_window_size(1440,900)
    driver.get("https://drugs.ncats.io/substances")

    driver.implicitly_wait(2)    
    driver.find_element(By.XPATH,"/html/body/div[2]/nav/div[1]/div[2]/ul/li[1]/form/div/input").click()
    driver.implicitly_wait(2)
    input_element = driver.find_element_by_css_selector('#search')
    driver.implicitly_wait(2)    
    action = ActionChains(driver)
    driver.implicitly_wait(2)
    action.move_to_element(input_element)
    driver.implicitly_wait(2)
    action.click(input_element)
    driver.implicitly_wait(2)    
    action.send_keys(keyword).perform()
    driver.implicitly_wait(2)
    driver.find_element(By.XPATH,"/html/body/div[2]/nav/div[1]/div[2]/ul/li[1]/form/div/span/div/button").click()    
    driver.implicitly_wait(60)
    driver.find_element_by_css_selector("#card-title > h3").click()
    driver.implicitly_wait(30)
    currentURL = driver.current_url
    driver.close()
    page = requests.get(currentURL)
    soup = BeautifulSoup(page.content, 'html.parser')
    route1 = soup.find_all('div',{'class':'subsub-title'})[0].find_all('span')[0].text
    if route1 is not None:
      route2 = route1.replace("\n","")
      route = route2.replace(" ","")
    else:
      route = ''
    return route

Test

In [21]:
from urllib.request import urlopen
from chembl_webresource_client.new_client import new_client
DBstring = "DB"
CHEMBLstring = "CHEMBL"

## Read excel file containing drug names
file = './COVID19_candidate_approved_drugs_combined_mti_pharm_targets.v0.xlsx'
df_c = pd.read_excel(file, sheet_name=0)
# df_6 = pd.read_excel(file, sheet_name=1)
# df_24 = pd.read_excel(file, sheet_name=2)

## Individual ID list
commonlist= df_c.iloc[:, 0].tolist()
# sixlist= df_6.iloc[:, 0].tolist()
# tflist= df_24.iloc[:, 0].tolist()
# # print(commonlist[0:40])

dfa=df_c
dfa.columns.values[0]="ID"

# SearchedIDList = []

# for i in commonlist:
#   if CHEMBLstring not in i:  
#     ## ChEMBL ID
#     searchedID=searchchemblid(i)
#     if searchedID=="NA":
#         searchedID==i

#     ## Developer 
#     # developer=searchdeveloper(i)
#     # DeveloperList.append(developer)

#   else:
#     ## ChEMBL ID
#     searchedID = i
#     SearchedIDList.append(searchedID)


# print(SearchedIDList)
# # print(commonlist)
  

FileNotFoundError: ignored

In [0]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore", 'This pattern has match groups')
DBstring = "DB"
CHEMBLstring = "CHEMBL" 
file = './COVID19_candidate_approved_drugs_combined.v1.xlsx'
df_c = pd.read_excel(file, sheet_name=0)
# df_6 = pd.read_excel(file, sheet_name=1)
# df_24 = pd.read_excel(file, sheet_name=2)
dfa=df_c
# dfb=df_6
# dfc=df_24

dfa.columns.values[0]="ID"
dfa['SearchedIDList']=dfa['ID'].apply(lambda x: x if CHEMBLstring in x else searchchemblid(x))
dfa['Indication']=dfa['SearchedIDList'].apply(lambda x : chemblindication(x) if CHEMBLstring in x else '')
dfa['Indication Paragraph']=dfa['ID'].apply(lambda x : searchDBIndication(x) if DBstring in x else '')
dfa['COVID-19 Clinical Trials']=dfa['name'].apply(trialsearch)
dfa['Developer']=dfa['ID'].apply(lambda x: searchdeveloper(x) if DBstring in x else '')
dfa['Route']=dfa['ID'].apply(lambda x: searchrouteDB(x) if DBstring in x else pd.isnull)
dfa['Route'] = dfa.apply(lambda row: searchrouteNIH(row['name']) if pd.isnull(row['Route']) else row['Route'],axis=1)
# # dfa['COVID-19 In Vitro']=dfa['name'].apply(invitrostudysearch)
# # dfa['COVID-19 In Silico']=dfa['name'].apply(insilicostudysearch)

# dfb.columns.values[0]="ID"
# dfb['SearchedIDList']=dfb['ID'].apply(lambda x: x if CHEMBLstring in x else searchchemblid(x))
# dfb['Indication']=dfb['SearchedIDList'].apply(lambda x : chemblindication(x) if CHEMBLstring in x else '')
# dfb['Indication Paragraph']=dfb['SearchedIDList'].apply(lambda x : searchDBIndication(x) if DBstring in x else '')
# dfb['COVID-19 Clinical Trials']=dfb['name'].apply(trialsearch)
# dfb['Developer']=dfa['ID'].apply(lambda x: searchdeveloper(x) if DBstring in x else '')
# dfb['Route']=dfb['ID'].apply(lambda x: searchrouteDB(x) if DBstring in x else pd.isnull)
# dfb['Route'] = dfb.apply(lambda row: searchrouteNIH(row['name']) if pd.isnull(row['Route']) else row['Route'],axis=1)
# # # # dfb['COVID-19 In Vitro']=dfb['name'].apply(invitrostudysearch)
# # # # dfb['COVID-19 In Silico']=dfb['name'].apply(insilicostudysearch)

# dfc.columns.values[0]="ID"
# dfc['SearchedIDList']=dfc['ID'].apply(lambda x: x if CHEMBLstring in x else searchchemblid(x))
# dfc['Indication']=dfc['SearchedIDList'].apply(lambda x : chemblindication(x) if CHEMBLstring in x else '')
# dfc['Indication Paragraph']=dfc['SearchedIDList'].apply(lambda x : searchDBIndication(x) if DBstring in x else '')
# dfc['COVID-19 Clinical Trials']=dfc['name'].apply(trialsearch)
# dfc['Developer']=dfc['ID'].apply(lambda x: searchdeveloper(x) if DBstring in x else '')
# dfc['Route']=dfc['ID'].apply(lambda x: searchrouteDB(x) if DBstring in x else pd.isnull)
# dfc['Route'] = dfc.apply(lambda row: searchrouteNIH(row['name']) if pd.isnull(row['Route']) else row['Route'],axis=1)
# # # # dfc['COVID-19 In Vitro']=dfc['name'].apply(invitrostudysearch)
# # # # dfc['COVID-19 In Silico']=dfc['name'].apply(insilicostudysearch)

writer = pd.ExcelWriter('COVID19_candidate_approved_drugs_combined_mti_pharm_targets_winnie.v1.xlsx', engine='xlsxwriter')
dfa.to_excel(writer, sheet_name='')
# dfb.to_excel(writer, sheet_name='Sheet2')
# dfc.to_excel(writer, sheet_name='Sheet3')
writer.save()
